In [1]:
import numpy as np
from collections import defaultdict
import nltk 
import pandas as pd
import os 
import json
import io
import gzip

The code shown below is from DSC3 Datathon

In [10]:
import random


ModuleNotFoundError: No module named 'xrange'

Fastest random sampler:
- Source: http://metadatascience.com/2014/02/27/random-sampling-from-very-large-files/

In [31]:
random.seed(42)
def random_sampler(filename, k):
	sample = []
	with open(filename, 'rb') as f:
		f.seek(0, 2)
		filesize = f.tell()
		random_set = sorted(random.sample(range(filesize), k))

		for i in range(k):
			f.seek(random_set[i])
			# Skip current line (because we might be in the middle of a line) 
			f.readline()
			# Append the next line to the sample set 
			sample.append(f.readline().rstrip())

	return sample

In [32]:
places2 = random_sampler("places.clean.json", 2000)


In [42]:
pd.DataFrame([sub.split(",") for sub in places2])

TypeError: a bytes-like object is required, not 'str'

In [19]:
%%time
#parsing in the data, takes time
places = random_sampler("places.clean.json", 200000)
users = random_sampler("users.clean.json", 200000)
reviews = random_sampler("reviews.clean.json", 200000)

Wall time: 1min 51s


In [20]:
#making the dict into dataframes
places = pd.DataFrame(places, del)
users = pd.DataFrame(users)
reviews = pd.DataFrame(reviews)

In [24]:
places.iloc[0]

0    b"{'name': u'LA ROMA', 'price': None, 'address...
Name: 0, dtype: object

In [22]:
reviews.head()

,0
0,"b""{'rating': 4.0, 'reviewerName': u'\\u68ee\\u..."
1,"b""{'rating': 4.0, 'reviewerName': u'ilhem ily'..."
2,"b""{'rating': 4.0, 'reviewerName': u'Sadanand A..."
3,"b""{'rating': 2.0, 'reviewerName': u'Marce Cast..."
4,"b""{'rating': 5.0, 'reviewerName': u'Marce Cast..."


In [23]:
users.head()

,0
0,"b""{'userName': u'Jacquelyn Dorris', 'jobs': [[..."
1,"b""{'userName': u'isaqeu jesus', 'jobs': None, ..."
2,"b""{'userName': u'Jason Wagner', 'jobs': None, ..."
3,"b""{'userName': u'Hermes Luca', 'jobs': None, '..."
4,"b""{'userName': u'Ahmad Syameel Anuar', 'jobs':..."


In [8]:
full = reviews.merge(users, how = 'outer') 
full.head()

,categories,gPlusPlaceId,gPlusUserId,rating,reviewText,reviewTime,reviewerName,unixReviewTime,currentPlace,education,jobs,previousPlaces,userName
0,[Giải Trí - Café],108103314380004200232,100000010817154263736,3.0,Chất lượng tạm ổn,"Jul 1, 2013",an lam,1.372687e+09,"[Thành phố Hồ Chí Minh, Việt Nam, [[], 1082309...","[[[], [], [], [], [], 6], [[Đại học Kiến trúc ...","[[Tổng công ty IDICO, Chuyên viên Kỹ thuật XD,...","[[tp. Nam Định, Nam Định, Việt Nam, [[], 20420...",an lam
1,[Turkish Cuisine],102194128241608748649,100000013500285534661,5.0,Wc si temiz duzenli..,"Jul 21, 2012",HALİL TURGUT,1.342871e+09,"[Adana, [[], 370000000, 353213330, 1]]","[[[], [], [], [], [], 6], [[Anadolu University...","[[Mustafa Nevzat İlaç San. A.Ş., Rebrezant, []...",None,HALİL TURGUT
2,"[Fishing, Pond Fish Supplier, Seafood Market]",101409858828175402384,100000021336848867366,5.0,何回も私は予定に休みがセルバに行ったので覚えて見て、分かります❗,"Jan 25, 2014",森田さとこ,1.390654e+09,None,"[[[], [], [], [], [], 6], []]",None,None,森田さとこ
3,[Museum],101477177500158511502,100000021336848867366,5.0,今度は予定に休みが登米市に行きたい❗☀😅🌌 楽しいに日帰りに登米の見学の観光(*^)(*^-...,"Jan 8, 2014",森田さとこ,1.389188e+09,None,"[[[], [], [], [], [], 6], []]",None,None,森田さとこ
4,[Police],106994170641063333085,100000021336848867366,4.0,気仙沼警察署に移転中に絆 👮🐎☺🙋🚓頑張ろう❗,"Jan 23, 2014",森田さとこ,1.390486e+09,None,"[[[], [], [], [], [], 6], []]",None,None,森田さとこ


Using Google's API to translate text.

In [9]:
import google.cloud.translate 

https://stackoverflow.com/questions/45501082/set-google-application-credentials-in-python-project-to-use-google-api

In [20]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="Datathon-ad4a6dac1782.json"

In [23]:
# Imports the Google Cloud client library
from google.cloud import translate

# Instantiates a client
translate_client = translate.Client()

# The text to translate
text = u'很多料加在同一碗粥裡，賣的很貴，但味道根本不搭。'
# The target language
target = 'en'

# Translates some text into Russian
translation = translate_client.translate(
    text,
    target_language=target)

print(u'Text: {}'.format(text))
print(u'Translation: {}'.format(translation['translatedText']))

Text: 很多料加在同一碗粥裡，賣的很貴，但味道根本不搭。
Translation: A lot of materials are added to the same bowl of porridge, which is very expensive to sell, but the taste is not at all.


In [24]:
reviews

,categories,gPlusPlaceId,gPlusUserId,rating,reviewText,reviewTime,reviewerName,unixReviewTime
0,[Giải Trí - Café],108103314380004200232,100000010817154263736,3.0,Chất lượng tạm ổn,"Jul 1, 2013",an lam,1.372687e+09
1,[Turkish Cuisine],102194128241608748649,100000013500285534661,5.0,Wc si temiz duzenli..,"Jul 21, 2012",HALİL TURGUT,1.342871e+09
2,"[Fishing, Pond Fish Supplier, Seafood Market]",101409858828175402384,100000021336848867366,5.0,何回も私は予定に休みがセルバに行ったので覚えて見て、分かります❗,"Jan 25, 2014",森田さとこ,1.390654e+09
3,[Museum],101477177500158511502,100000021336848867366,5.0,今度は予定に休みが登米市に行きたい❗☀😅🌌 楽しいに日帰りに登米の見学の観光(*^)(*^-...,"Jan 8, 2014",森田さとこ,1.389188e+09
4,[Police],106994170641063333085,100000021336848867366,4.0,気仙沼警察署に移転中に絆 👮🐎☺🙋🚓頑張ろう❗,"Jan 23, 2014",森田さとこ,1.390486e+09
5,[Theme Park],107443380765688783479,100000021336848867366,5.0,久しぶりに仙台市八木山ベニーランドに乗りたい❗,"Jan 19, 2014",森田さとこ,1.390193e+09
6,[Shopping Mall],107998775704210253660,100000021336848867366,5.0,私は今度は休みがザ・モール★の映画に行って来ます。,"Jan 27, 2014",森田さとこ,1.390826e+09
7,[Art Museum],111367239248458923884,100000021336848867366,4.0,イベントの光のページェント が素晴らしい❗🔍 私はとても仮面ライダーが素晴らしいが良いがです🎵ネ。,"Jan 22, 2014",森田さとこ,1.390446e+09
8,[Science Museum],111951491217608644315,100000021336848867366,4.0,ヒマ👽休みが仙台市科学館 に行って見ます❗,"Jan 22, 2014",森田さとこ,1.390388e+09
9,[Grocery Store],113899620351618240699,100000021336848867366,5.0,久しぶりに青葉城に行きたい❗,"Feb 13, 2014",森田さとこ,1.392295e+09
